Sequence Models / NLP
Amaç

Metin tabanlı modelleri (LSTM, GRU, Transformers) uygulayıp sonuçlarını kıyaslamak.

Görev Adımları

Veri Hazırlığı

Kaggle’dan IMDB Sentiment Analysis dataset’ini indir.

Train/test split hazır (varsa tekrar düzenle).

Tokenization ve Padding uygula.

Model 1 – LSTM / GRU

Embedding Layer (örn. 100 boyutlu).

LSTM veya GRU katmanı ekle.

Çıkış: Sigmoid (binary classification).

Model 2 – Transformer Tabanlı

HuggingFace’den BERT veya RoBERTa pre-trained modeli al.

LoRA veya PEFT ile fine-tune et.

Değerlendirme

Accuracy, Precision, Recall, F1 Score hesapla.

BLEU Score (opsiyonel, sequence generation senaryosu için).

Modelleri yan yana kıyasla.

📌 Beklenen Çıktılar: F1 Score tablosu, confusion matrix, model karşılaştırma raporu.

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Veri Hazırlığı
- Veri seti text ve label olarak sütunlara ayrıldı 
- "train_test_split" ile %80 train %20 test olarak bölündü.
- stratify = y ile train ve test setlerinde sınıf dağılımı orjinal veri ile aynı tutuldu yani label 1 - 0 sayıları dengeli
- random_state = 42 ile split işlemi tekrarlanabilir yani her çalıştırıldığında aynı train/test bölünmesi elde edilmiş olur

In [8]:
df = pd.read_csv("../data/movie.csv")
X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, stratify = y, random_state = 42
)

**HuggingFace + PyTorch**
- Pre-trained tokenizer ile metinler token ID'lerine çevrildi,
- Padding ve truncation ile tüm diziler aynı uzunlukta getirildi,
- Labels tensor formatına dönüştürüldü,
- DataLoader için hazır hale getirildi

In [11]:
from transformers import AutoTokenizer
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(list(X_train), padding=True, truncation=True, max_length=256, return_tensors="pt")
test_encodings = tokenizer(list(X_test), padding=True, truncation=True, max_length=256, return_tensors="pt")

y_train_tensor = torch.tensor(list(y_train.values))
y_test_tensor = torch.tensor(list(y_test.values))

from torch.utils.data import TensorDataset, DataLoader #https://docs.pytorch.org/docs/stable/data.html#module-torch.utils.data

train_dataset = TensorDataset(train_encodings['input_ids'], y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)



## LSTM / GRU Model Katmanı